# DataFrames

In [1]:
from pyspark.sql import SparkSession

# create a spark session
spark_session = SparkSession.builder.getOrCreate()

# reading a csv file
df = spark_session.read.csv("sample_data/301-people-info.csv", header=True)

# register in the catalog
df.createOrReplaceTempView("person_info")

df.show()

+------------------+--------+--------------------+-----------+-----+----------+------+
|              name| surname|              street|       city|state|postalcode|salary|
+------------------+--------+--------------------+-----------+-----+----------+------+
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
+------------------+--------+--------------------+-----------+-----+----------+------+



## Queries in a DataFrame

### Select types

```.select(*cols)``` select columns and return a new DataFrame

Show entries in the ```name``` column:

In [2]:
df.select("name").show()

+------------------+
|              name|
+------------------+
|              John|
|              Jack|
|       John Da Man|
|           Stephen|
|              null|
|Joan the bone Anne|
|              Jack|
+------------------+



Show entries in the ```name``` and ```salary + 1000``` column:

In [3]:
df.select(df["name"],df["salary"] + 1000).show()

+------------------+---------------+
|              name|(salary + 1000)|
+------------------+---------------+
|              John|        11000.0|
|              Jack|        19900.0|
|       John Da Man|        33100.0|
|           Stephen|        33908.0|
|              null|        45221.0|
|Joan the bone Anne|        14900.0|
|              Jack|        19900.0|
+------------------+---------------+



Show entries in the ```name```, ```salary``` and ```salary > 20000``` column:

In [4]:
df.select("name", "salary", df['salary'] > 20000).show()

+------------------+------+----------------+
|              name|salary|(salary > 20000)|
+------------------+------+----------------+
|              John| 10000|           false|
|              Jack| 18900|           false|
|       John Da Man| 32100|            true|
|           Stephen| 32908|            true|
|              null| 44221|            true|
|Joan the bone Anne| 13900|           false|
|              Jack| 18900|           false|
+------------------+------+----------------+



In [5]:
from pyspark.sql import Row
from pyspark.sql import functions as f
df1 = spark_session.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b", "c": "d"}),
                                     Row(a=2, intlist=[3,4,5], mapfield={"a": "c", "c": "b"})])

In this example we are using ```.select()``` with ```pyspark.sql.functions.explode()```. This is useful when you have an array or a map in a row.

In [6]:
df1.select("a", f.explode(df1.mapfield).alias("key", "value")).show()

+---+---+-----+
|  a|key|value|
+---+---+-----+
|  1|  a|    b|
|  1|  c|    d|
|  2|  a|    c|
|  2|  c|    b|
+---+---+-----+



In [7]:
df1.select("a", f.explode(df1.intlist).alias("anInt")).show()

+---+-----+
|  a|anInt|
+---+-----+
|  1|    1|
|  1|    2|
|  1|    3|
|  2|    3|
|  2|    4|
|  2|    5|
+---+-----+



### When clause

Evaluate one or a list of conditions. ```.otherwise()``` is None as default.

In [8]:
df2 = df.select("name", f.when(df.state == "PA", df.state).otherwise(None).alias("state"))
df2.show()

+------------------+-----+
|              name|state|
+------------------+-----+
|              John| null|
|              Jack|   PA|
|       John Da Man| null|
|           Stephen| null|
|              null| null|
|Joan the bone Anne| null|
|              Jack|   PA|
+------------------+-----+



```.dropna()``` to drop null values.

In [9]:
df2.dropna(subset="state").show()

+----+-----+
|name|state|
+----+-----+
|Jack|   PA|
|Jack|   PA|
+----+-----+



```.isin()``` checks if the values are in a column

In [10]:
df[df["name"].isin("John Da Man", "Jack")].show()

+-----------+--------+-----------------+---------+-----+----------+------+
|       name| surname|           street|     city|state|postalcode|salary|
+-----------+--------+-----------------+---------+-----+----------+------+
|       Jack|McGinnis|     220 hobo Av.|    Phila|   PA|     09119| 18900|
|John Da Man|  Repici|120 Jefferson St.|Riverside|   NJ|     08075| 32100|
|       Jack|McGinnis|     220 hobo Av.|    Phila|   PA|     09119| 18900|
+-----------+--------+-----------------+---------+-----+----------+------+



### Like

```.like()``` is a binary operator that verifies if each row in the column contains "McGinnis"

In [11]:
 df.select("name", df["surname"].like("McGinnis")).show()

+------------------+---------------------+
|              name|surname LIKE McGinnis|
+------------------+---------------------+
|              John|                false|
|              Jack|                 true|
|       John Da Man|                false|
|           Stephen|                false|
|              null|                false|
|Joan the bone Anne|                false|
|              Jack|                 true|
+------------------+---------------------+



### Startswith

```.startswith()``` is a binary operator that verifies if each row in the column starts with "S"

In [12]:
df.select("name", df["name"].startswith("S")).show()

+------------------+-------------------+
|              name|startswith(name, S)|
+------------------+-------------------+
|              John|              false|
|              Jack|              false|
|       John Da Man|              false|
|           Stephen|               true|
|              null|               null|
|Joan the bone Anne|              false|
|              Jack|              false|
+------------------+-------------------+



```.endswith()``` is a binary operator that verifies if each row in the column ends with "n"

In [13]:
df.select("name", df["name"].endswith("n")).show()

+------------------+-----------------+
|              name|endswith(name, n)|
+------------------+-----------------+
|              John|             true|
|              Jack|            false|
|       John Da Man|             true|
|           Stephen|             true|
|              null|             null|
|Joan the bone Anne|            false|
|              Jack|            false|
+------------------+-----------------+



```.substr(a, b)``` returns a substring between a and b from each row in a given column.

In [14]:
df.select(df["name"].substr(0,5)).show()

+---------------------+
|substring(name, 0, 5)|
+---------------------+
|                 John|
|                 Jack|
|                John |
|                Steph|
|                 null|
|                Joan |
|                 Jack|
+---------------------+



```.between(a, b)``` returns if the rows in a given column contains values between a and b.

In [15]:
df.select("salary", df["salary"].between(30000, 40000)).show()

+------+-----------------------------------------+
|salary|((salary >= 30000) AND (salary <= 40000))|
+------+-----------------------------------------+
| 10000|                                    false|
| 18900|                                    false|
| 32100|                                     true|
| 32908|                                     true|
| 44221|                                    false|
| 13900|                                    false|
| 18900|                                    false|
+------+-----------------------------------------+



In [16]:
df.select("state", df["state"].between("A", "O")).show()

+-----+-------------------------------+
|state|((state >= A) AND (state <= O))|
+-----+-------------------------------+
|   NJ|                           true|
|   PA|                          false|
|   NJ|                           true|
|   SD|                          false|
|   SD|                          false|
|   CO|                           true|
|   PA|                          false|
+-----+-------------------------------+



### groupby

```.groupBy(*cols)``` aggregates specified columns

In [17]:
df.groupBy("state").count().show()

+-----+-----+
|state|count|
+-----+-----+
|   NJ|    2|
|   PA|    2|
|   SD|    2|
|   CO|    1|
+-----+-----+



```.filter(condition)``` filter rows using the given condition. It's the same as the SQL ```WHERE``` clause.

In [18]:
df.filter(df["state"] > "O").show()

+-------+--------+--------------------+--------+-----+----------+------+
|   name| surname|              street|    city|state|postalcode|salary|
+-------+--------+--------------------+--------+-----+----------+------+
|   Jack|McGinnis|        220 hobo Av.|   Phila|   PA|     09119| 18900|
|Stephen|   Tyler|7452 Terrace At t...|SomeTown|   SD|     91234| 32908|
|   null|Blankman|                null|SomeTown|   SD|     00298| 44221|
|   Jack|McGinnis|        220 hobo Av.|   Phila|   PA|     09119| 18900|
+-------+--------+--------------------+--------+-----+----------+------+



### Sorting data

The ```.sort()``` is used for ordering specified columns. It's also used ```.orderBy()```.

In [19]:
 df.sort("salary", ascending=False).show()

+------------------+--------+--------------------+-----------+-----+----------+------+
|              name| surname|              street|       city|state|postalcode|salary|
+------------------+--------+--------------------+-----------+-----+----------+------+
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|
+------------------+--------+--------------------+-----------+-----+----------+------+



In [20]:
df.sort(df["salary"].asc()).show()

+------------------+--------+--------------------+-----------+-----+----------+------+
|              name| surname|              street|       city|state|postalcode|salary|
+------------------+--------+--------------------+-----------+-----+----------+------+
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|
+------------------+--------+--------------------+-----------+-----+----------+------+



In [21]:
df.sort(df["state"].desc()).show()

+------------------+--------+--------------------+-----------+-----+----------+------+
|              name| surname|              street|       city|state|postalcode|salary|
+------------------+--------+--------------------+-----------+-----+----------+------+
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|
+------------------+--------+--------------------+-----------+-----+----------+------+



In [22]:
df.orderBy(["city", "salary"],ascending=[0,1]).show()

+------------------+--------+--------------------+-----------+-----+----------+------+
|              name| surname|              street|       city|state|postalcode|salary|
+------------------+--------+--------------------+-----------+-----+----------+------+
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|
+------------------+--------+--------------------+-----------+-----+----------+------+



### Adding columns

Image our people received 20% of salary increase, but it's important to keep the old salary.

```.withColumn()``` returns a new DataFrame by adding or replacing when it has the same name.

In [23]:
df_temp = df.withColumn('sallary_increase', df.salary * 1.2)
df_temp.show()

+------------------+--------+--------------------+-----------+-----+----------+------+----------------+
|              name| surname|              street|       city|state|postalcode|salary|sallary_increase|
+------------------+--------+--------------------+-----------+-----+----------+------+----------------+
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|         12000.0|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|         22680.0|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|         38520.0|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|         39489.6|
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|         53065.2|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|         16680.0|
|              Jack|McGinnis|        220 hobo Av.|      Phila|  

### Updating columns

You notice you wrote the column ```salary_increase``` wrong.

```.withColumnRenamed()``` returns a new DataFrame by renaming an existing column.

In [24]:
df_temp = df_temp.withColumnRenamed('sallary_increase', 'salary_increase')
df_temp.show()

+------------------+--------+--------------------+-----------+-----+----------+------+---------------+
|              name| surname|              street|       city|state|postalcode|salary|salary_increase|
+------------------+--------+--------------------+-----------+-----+----------+------+---------------+
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|        12000.0|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|        22680.0|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|        38520.0|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|        39489.6|
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|        53065.2|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|        16680.0|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     

### Removing columns

However, your boss said you to drop salary_increase column.

```.drop()``` return a new DataFrame removing the specified column.

In [25]:
df_temp = df_temp.drop("salary_increase")

In [26]:
df_temp.show()

+------------------+--------+--------------------+-----------+-----+----------+------+
|              name| surname|              street|       city|state|postalcode|salary|
+------------------+--------+--------------------+-----------+-----+----------+------+
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|     08075| 10000|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|     08075| 32100|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|     91234| 32908|
|              null|Blankman|                null|   SomeTown|   SD|     00298| 44221|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|     00123| 13900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|     09119| 18900|
+------------------+--------+--------------------+-----------+-----+----------+------+



## Query as SQL

In [27]:
query = "Select * from person_info where salary > 20000"

spark_session.sql(query).show()

+-----------+--------+--------------------+---------+-----+----------+------+
|       name| surname|              street|     city|state|postalcode|salary|
+-----------+--------+--------------------+---------+-----+----------+------+
|John Da Man|  Repici|   120 Jefferson St.|Riverside|   NJ|     08075| 32100|
|    Stephen|   Tyler|7452 Terrace At t...| SomeTown|   SD|     91234| 32908|
|       null|Blankman|                null| SomeTown|   SD|     00298| 44221|
+-----------+--------+--------------------+---------+-----+----------+------+



## Output

### DF to RDD

In [28]:
rdd = df.rdd
rdd.collect()

[Row(name='John', surname='Doe', street='120 jefferson st.', city='Riverside', state='NJ', postalcode='08075', salary='10000'),
 Row(name='Jack', surname='McGinnis', street='220 hobo Av.', city='Phila', state='PA', postalcode='09119', salary='18900'),
 Row(name='John Da Man', surname='Repici', street='120 Jefferson St.', city='Riverside', state='NJ', postalcode='08075', salary='32100'),
 Row(name='Stephen', surname='Tyler', street='7452 Terrace At the Plaza road', city='SomeTown', state='SD', postalcode='91234', salary='32908'),
 Row(name=None, surname='Blankman', street=None, city='SomeTown', state='SD', postalcode='00298', salary='44221'),
 Row(name='Joan the bone Anne', surname='Jet', street='9th at Terrace plc', city='Desert City', state='CO', postalcode='00123', salary='13900'),
 Row(name='Jack', surname='McGinnis', street='220 hobo Av.', city='Phila', state='PA', postalcode='09119', salary='18900')]

### DF to RDD of string

In [29]:
df.toJSON().take(2)

['{"name":"John","surname":"Doe","street":"120 jefferson st.","city":"Riverside","state":"NJ","postalcode":"08075","salary":"10000"}',
 '{"name":"Jack","surname":"McGinnis","street":"220 hobo Av.","city":"Phila","state":"PA","postalcode":"09119","salary":"18900"}']

### DF to Pandas DataFrame

In [30]:
df.toPandas()

,name,surname,street,city,state,postalcode,salary
0,John,Doe,120 jefferson st.,Riverside,NJ,08075,10000
1,Jack,McGinnis,220 hobo Av.,Phila,PA,09119,18900
2,John Da Man,Repici,120 Jefferson St.,Riverside,NJ,08075,32100
3,Stephen,Tyler,7452 Terrace At the Plaza road,SomeTown,SD,91234,32908
4,None,Blankman,None,SomeTown,SD,00298,44221
5,Joan the bone Anne,Jet,9th at Terrace plc,Desert City,CO,00123,13900
6,Jack,McGinnis,220 hobo Av.,Phila,PA,09119,18900


## Saving DataFrame

In [31]:
df.write.save("sample_data/302-person-info.parquet")

In [32]:
df.write.save("sample_data/302-person-info.json", format="json")

## Stopping Spark Session

In [33]:
spark_session.stop()